In [39]:
from kiteconnect import KiteConnect
from datetime import datetime,timedelta
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime

In [40]:
kite = KiteConnect(api_key="3np0d05xqq02k905")
kite.set_access_token("uS1z1psPljc3OOlgDQLeUjWMEifASGHZ")

In [41]:
# df_instruments=pd.DataFrame(kite.instruments())
# df_instruments.to_csv("instruments_all.csv", index=False)

In [42]:
def get_candle_fig(instrument_code,start,end,interval):
    df_historical=pd.DataFrame(kite.historical_data(instrument_code,start,end,interval))
    if len(df_historical)>0:
        fig = go.Figure(data=[go.Candlestick(x=df_historical['date'],
                        open=df_historical['open'],
                        high=df_historical['high'],
                        low=df_historical['low'],
                        close=df_historical['close'])])
        return (fig)
    else:
        return (None)
    
def get_candle(instrument_code,start,end,interval):
    df_historical=pd.DataFrame(kite.historical_data(instrument_code,start,end,interval))
    if len(df_historical)>0:
        return (df_historical)
    else:
        return (pd.DataFrame())

In [43]:
def get_candle_signal(instrument_code,sample_size):
    try:
        df_summary=pd.DataFrame()
        for i in range(sample_size):
            start=((datetime.now()+timedelta(days=-i)).strftime("%Y-%m-%d"))+" 09:00:00"
            end=((datetime.now()+timedelta(days=-i)).strftime("%Y-%m-%d"))+" 16:00:00"
            interval="5minute"
            df_historical=get_candle(instrument_code,start,end,interval)
            
            if len(df_historical)>0:
                df_historical['change']=df_historical['close']-df_historical['open']
                df_historical['candle_type']=df_historical['change'].apply(lambda x:'green' if x>0 else 'red')
                try:
                    candle_ratio= ((df_historical['candle_type'].value_counts()['green'])/(df_historical['candle_type'].count()))
                except:
                    candle_ratio=0
                df_summary_dict={'day':end,
                                'candle_ratio':candle_ratio}
                df_temp=pd.DataFrame([df_summary_dict])
                df_summary=pd.concat([df_summary,df_temp])
        return ((sum(df_summary['candle_ratio'])*2)/sample_size)
    except:
        return 0
    
def get_candle_signal_2(instrument_code,sample_size):
    try:
        df_summary=pd.DataFrame()
        for i in range(sample_size):
            start=((datetime.now()+timedelta(days=-i)).strftime("%Y-%m-%d"))+" 09:00:00"
            end=((datetime.now()+timedelta(days=-i)).strftime("%Y-%m-%d"))+" 16:00:00"
            interval="5minute"
            df_historical=get_candle(instrument_code,start,end,interval)
            
            if len(df_historical)>0:
                df_historical['change']=df_historical['close']-df_historical['open']
                df_historical['candle_type']=df_historical['change'].apply(lambda x:'green' if x>0 else 'red')
                try:
                    candle_ratio= ((df_historical['candle_type'].value_counts()['green'])/(df_historical['candle_type'].count()))
                except:
                    candle_ratio=0
                df_summary_dict={'day':end,
                                'candle_ratio':candle_ratio}
                df_temp=pd.DataFrame([df_summary_dict])
                df_summary=pd.concat([df_summary,df_temp])
        return (len(df_summary[df_summary['candle_ratio']>0.5])/len(df_summary))
    except:
        return 0
    
def get_candle_signal_3(instrument_code,sample_size):
    try:
        df_summary=pd.DataFrame()
        for i in range(sample_size):
            start=((datetime.now()+timedelta(days=-i)).strftime("%Y-%m-%d"))+" 09:00:00"
            end=((datetime.now()+timedelta(days=-i)).strftime("%Y-%m-%d"))+" 16:00:00"
            interval="5minute"
            df_historical=get_candle(instrument_code,start,end,interval)
            
            if len(df_historical)>0:
                df_historical['change']=df_historical['close']-df_historical['open']
                df_historical['candle_type']=df_historical['change'].apply(lambda x:1 if x>0 else -1)
                df_historical['candle_weight']=df_historical['change'].abs()/df_historical['change'].abs().max()
                df_historical['candle_strength']=df_historical['candle_type']*df_historical['candle_weight']
                try:
                    candle_ratio= sum(df_historical['candle_strength'])
                except:
                    candle_ratio=0
                df_summary_dict={'day':end,
                                'candle_ratio':candle_ratio}
                df_temp=pd.DataFrame([df_summary_dict])
                df_summary=pd.concat([df_summary,df_temp])
        return ((sum(df_summary['candle_ratio']))/sample_size)
    except:
        return 0

In [44]:
df_instruments1=pd.read_excel('final_instruments.xlsx')
df_instruments1['signal_3_1d']=df_instruments1['instrument_token'].apply(lambda x: get_candle_signal_3(x,1))
df_instruments1['signal_3_7d']=df_instruments1['instrument_token'].apply(lambda x: get_candle_signal_3(x,7))
df_instruments1['signal_3_31d']=df_instruments1['instrument_token'].apply(lambda x: get_candle_signal_3(x,31))
df_instruments1=df_instruments1.sort_values(by="signal_3_31d",ascending=False).reset_index(drop=True)

In [46]:
date=datetime.now().strftime("%d_%m_%y")
df_instruments1.to_csv(f'weighted_final_shortlist_{date}.csv',index=False)

In [51]:
ce_shortlist_df=df_instruments1[(df_instruments1['signal_3_1d']>0)&(df_instruments1['signal_3_7d']>0)&(df_instruments1['signal_3_31d']>0)]
pe_shortlist_df=df_instruments1[(df_instruments1['signal_3_1d']<0)&(df_instruments1['signal_3_7d']<0)&(df_instruments1['signal_3_31d']<0)]

In [53]:
ce_shortlist_df.sort_values()

In [37]:
sample_size=7
instrument_code=895745
df_summary=pd.DataFrame()
for i in range(sample_size):
    start=((datetime.now()+timedelta(days=-i)).strftime("%Y-%m-%d"))+" 09:00:00"
    end=((datetime.now()+timedelta(days=-i)).strftime("%Y-%m-%d"))+" 16:00:00"
    interval="5minute"
    df_historical=get_candle(instrument_code,start,end,interval)
    
    if len(df_historical)>0:
        df_historical['change']=df_historical['close']-df_historical['open']
        df_historical['candle_type']=df_historical['change'].apply(lambda x:1 if x>0 else -1)
        df_historical['candle_weight']=df_historical['change'].abs()/df_historical['change'].abs().max()
        df_historical['candle_strength']=df_historical['candle_type']*df_historical['candle_weight']
        try:
            candle_ratio= sum(df_historical['candle_strength'])
        except:
            candle_ratio=0
        df_summary_dict={'day':end,
                        'candle_ratio':candle_ratio}
        df_temp=pd.DataFrame([df_summary_dict])
        df_summary=pd.concat([df_summary,df_temp])
        df_summary.dropna(subset="candle_ratio",inplace=True)
print ((sum(df_summary['candle_ratio']))/sample_size)

1.203416149068343


In [ ]:

# Place an order
order_id = kite.place_order(tradingsymbol="INFY",
                            exchange=kite.EXCHANGE_NSE,
                            transaction_type=kite.TRANSACTION_TYPE_BUY,
                            quantity=1,
                            variety=kite.VARIETY_AMO,
                            order_type=kite.ORDER_TYPE_MARKET,
                            product=kite.PRODUCT_CNC,
                            validity=kite.VALIDITY_DAY)



# Fetch all orders
kite.orders()

# Get instruments
kite.instruments()

# Place an mutual fund order
kite.place_mf_order(
    tradingsymbol="INF090I01239",
    transaction_type=kite.TRANSACTION_TYPE_BUY,
    amount=5000,
    tag="mytag"
)

# Cancel a mutual fund order
kite.cancel_mf_order(order_id="order_id")

# Get mutual fund instruments
kite.mf_instruments()

In [ ]:
a=-1
a.abs()

In [ ]:
###############################################################################
#
# The MIT License (MIT)
#
# Copyright (c) Zerodha Technology Pvt. Ltd.
#
# This example shows how to run KiteTicker in threaded mode.
# KiteTicker runs in seprate thread and main thread is blocked to juggle between
# different modes for current subscribed tokens. In real world web apps
# the main thread will be your web server and you can access WebSocket object
# in your main thread while running KiteTicker in separate thread.
###############################################################################

import time
import logging
from kiteconnect import KiteTicker

logging.basicConfig(level=logging.DEBUG)

# Initialise.
kws = KiteTicker("your_api_key", "your_access_token")

# RELIANCE BSE
tokens = [738561]


# Callback for tick reception.
def on_ticks(ws, ticks):
    if len(ticks) > 0:
        logging.info("Current mode: {}".format(ticks[0]["mode"]))


# Callback for successful connection.
def on_connect(ws, response):
    logging.info("Successfully connected. Response: {}".format(response))
    ws.subscribe(tokens)
    ws.set_mode(ws.MODE_FULL, tokens)
    logging.info("Subscribe to tokens in Full mode: {}".format(tokens))


# Callback when current connection is closed.
def on_close(ws, code, reason):
    logging.info("Connection closed: {code} - {reason}".format(code=code, reason=reason))


# Callback when connection closed with error.
def on_error(ws, code, reason):
    logging.info("Connection error: {code} - {reason}".format(code=code, reason=reason))


# Callback when reconnect is on progress
def on_reconnect(ws, attempts_count):
    logging.info("Reconnecting: {}".format(attempts_count))


# Callback when all reconnect failed (exhausted max retries)
def on_noreconnect(ws):
    logging.info("Reconnect failed.")


# Assign the callbacks.
kws.on_ticks = on_ticks
kws.on_close = on_close
kws.on_error = on_error
kws.on_connect = on_connect
kws.on_reconnect = on_reconnect
kws.on_noreconnect = on_noreconnect

# Infinite loop on the main thread.
# You have to use the pre-defined callbacks to manage subscriptions.
kws.connect(threaded=True)

# Block main thread
logging.info("This is main thread. Will change webosocket mode every 5 seconds.")

count = 0
while True:
    count += 1
    if count % 2 == 0:
        if kws.is_connected():
            logging.info("### Set mode to LTP for all tokens")
            kws.set_mode(kws.MODE_LTP, tokens)
    else:
        if kws.is_connected():
            logging.info("### Set mode to quote for all tokens")
            kws.set_mode(kws.MODE_QUOTE, tokens)

    time.sleep(5)
